# The Battle of Neighborhoods

## 1. Introduction: COFFEE ETHNIC IN DA NANG

In such a large and rich of coffee culture city like Da Nang, Viet Nam, it will be competitive to start up coffee business. In this case my contractor is a humble Vietnamese man who has contacted me to give advises and draw up essential lines of business prediction and back-up plans (but this part we will just discuss about predicting hot spot).

## 2. Orientation

First of all we need to collect Data of all coffee shops in Da Nang including their name, id, location (address, latitude, longitude) then pick up the "hot" neighbor where locates most of the venue. In order to asset Data we use FourSquare and apply folium for visualizing a particular neighbor in which that we will observe customer "traffic" and predict an appropriate location of new coffee shop in town. In this case you will find its temporary name on the folium map, "O Day Roi!"(Meaning "Here It Is!" in Vietnamese)

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

In [13]:
CLIENT_ID = 'my client id' # your Foursquare ID
CLIENT_SECRET = 'my client secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: my client id
CLIENT_SECRET:my client secret


In [3]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Da Nang",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [4]:
d = data.json()["response"]
d.keys()

dict_keys(['geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [5]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [6]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [7]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 40


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a26a41a31ac6c676705e94c',
  'name': 'cộng cà phê',
  'location': {'address': '39',
   'crossStreet': 'Nguyễn Thái Học',
   'lat': 16.068062879455486,
   'lng': 108.22351222071603,
   'labeledLatLngs': [{'label': 'display',
     'lat': 16.068062879455486,
     'lng': 108.22351222071603}],
   'postalCode': '551105',
   'cc': 'VN',
   'city': 'Đà Nẵng',
   'state': 'Thành Phố Đà Nẵng',
   'country': 'Việt Nam',
   'formattedAddress': ['39 (Nguyễn Thái Học)',
    'Đà Nẵng',
    'Thành Phố Đà Nẵng',
    'Việt Nam']},
  'categories': [{'id': '4bf58dd8d48988d1e0931735',
    'name': 'Coffee Shop',
    'pluralName': 'Coffee Shops',
    'shortName': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [12]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    # print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

found 7 cafes


,uid,name,shortname,address,postalcode,lat,lng
0,5a26a41a31ac6c676705e94c,cộng cà phê,Coffee Shop,,551105,16.068063,108.223512
1,5627c637498e21ca8f40f4be,Leaf Drinks and More,Coffee Shop,,550000,16.067020,108.224790
2,55ff7cf0498e203b2932d5fd,Namunamu Cafe 180,Café,,550000,16.066268,108.224496
3,51bd1671498e58791c2ea92d,Freeman Coffee,Coffee Shop,,550000,16.070851,108.221230
4,4c25fdd8f1272d7f33ac85c5,Highlands Coffee,Café,,60999,16.069946,108.224961


In [9]:
DaNang_center = d["geocode"]["center"]
DaNang_center

{'lat': 16.06778, 'lng': 108.22083}

In [10]:
from folium import plugins


map_DaNang = folium.Map(location=[16.06778, 108.22083], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_DaNang)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_DaNang.add_child(plugins.HeatMap(hm_data))

map_DaNang

## 3. Conclusion

We will need a location where we can catch out customers from "hot" location we have picked up from the map and stay in a certain distance so as to lessen the competivity of business.

In [11]:
lat = 16.06778
lng = 108.22083
map_DaNang = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_DaNang)
map_DaNang

Look at the blue buble, here we find out that it will locate about the crossroad between Pham Hong Thai street and Nguyen Chi Thanh street. As I know this neighborhood is safe and right at the center, it's off to the hot spot of night life and on the passing-by path of high school students in the area.